#### Installing Necessary Dependencies

In [5]:
# !pip install beautifulsoup4 requests selenium psycopg2

#### Importing Libraries

In [150]:
# Necessary imports
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
import requests
import re
import psycopg2

#### Testing for Response 200 to Establish Web scrapping

In [ ]:
url = 'https://www.aliexpress.com/w/wholesale-laptop.html?spm=a2g0o.home.search.0'
r = requests.get(url)

print(r)

#### Configure Selenium ChromeDriver Options and Test Run It

In [6]:
# Set up options and service
options = Options()
options.headless = True
service = Service(executable_path='chromedriver-mac-x64/chromedriver')

# Initialise the webdriver
driver = webdriver.Chrome(service=service, options=options)

# Define the URL
url = 'https://www.aliexpress.com/w/wholesale-laptop.html?spm=a2g0o.home.search.0'

# Use Selenium to open the page
# driver.get(url)

# Wait for dynamic content to load
time.sleep(10)

#Get the page source and close the browser
page_source = driver.page_source
driver.quit()

#### Data Extraction

In [7]:
# Configure Selenium ChromeDriver options
options = Options()
options.headless = True
service = Service(executable_path='chromedriver-mac-x64/chromedriver')

# Initialise the webdriver
driver = webdriver.Chrome(service=service, options=options)

# List of store extracted
product_name = []
store_name = []
store_link = []
actual_prices = []
disc_prices = []
discount_percent = []
shipping_prices = []
amount_saved = []
item_sold = []


# Assuming we know the total number of pages
total_pages = 60

for pages in range(1, total_pages + 1):
    url = f'https://www.aliexpress.com/w/wholesale-laptop.html?g=y&SearchText=laptop&page={pages}'
    driver.get(url)

    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'list--gallery--C2f2tvm')))
    except TimeoutException:
        print("Page took too long to load, skipping this page.")
        continue


    #time.sleep(4)

    # Lets use BeautifulSoup to parse the loaded page_source
    #Get the page source and close the browser
    #page_source = driver.page_source
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Adjust the class selectors based on current website structure
    laptops = soup.find_all('div', class_ = 'list--gallery--C2f2tvm search-item-card-wrapper-gallery')

    for laptop in laptops:
        # product_name
        try:
            product_name_item = laptop.find('div', class_ = 'multi--title--G7dOCj3').text
        except AttributeError:
            product_name_item = ''

        # discounted prices
        disc_prices_item = laptop.find('div', class_ = 'multi--price-sale--U-S0jtj').text.replace('GHS', '').replace(',', '')

        # actual prices
        try:
            actual_prices_item = laptop.find('div', class_ = 'multi--price-original--1zEQqOK').text.replace('GHS', '').replace(',', '')
            if actual_prices_item == '':
                actual_prices_item = disc_prices_item
        except AttributeError:
            actual_prices_item = disc_prices_item

        # discount percent
        try:
            discount_percent_item = laptop.find('span', class_ = 'multi--discount--3hksz5G').text.replace('-', '').replace('%', '')
            if discount_percent_item == '':
                discount_percent_item = '0.0'
        except AttributeError:
            discount_percent_item = '0.0'

        
        # saved_amount
        try:
            amount_saved_item = laptop.find('span', class_= 'tag--textStyle--3dc7wLU').text.replace('Save ', '').replace(',', '').replace('GHS', '')
            if amount_saved_item == '':
                amount_saved_item = '0.0'
        except AttributeError:
            amount_saved_item = ''


        # store name
        store_name_item = laptop.find('a', class_ = 'cards--storeLink--XkKUQFS').text

        # store link
        try:
            store_link_item = laptop.find('a', class_ = 'cards--storeLink--XkKUQFS')['href']
        except(TypeError, KeyError):
            store_link_item = ''

        # shipping prices
        try:
            shipping_price_item = laptop.find('span', class_ = 'tag--text--1BSEXVh tag--textStyle--3dc7wLU multi--serviceStyle--1Z6RxQ4').text.replace('GHS', '').replace(',', '')
        except AttributeError:
            shipping_price_item = ''


        # item sold
        try:
            item_sold_text = laptop.find('span', class_ = 'multi--trade--Ktbl2jB').text
            item_sold_value = item_sold_text.split()[0] if item_sold_text else ''
        except AttributeError:
            item_sold_value = ''

        
        # Append data to list
        product_name.append(product_name_item)
        disc_prices.append(disc_prices_item)
        store_name.append(store_name_item)
        store_link.append(store_link_item)
        shipping_prices.append(shipping_price_item)
        item_sold.append(item_sold_value)
        discount_percent.append(discount_percent_item)
        amount_saved.append(amount_saved_item)
        actual_prices.append(actual_prices_item)
        
# All information scrapped, quit the driver
driver.quit()

# Create a Dataframe
data = {
    'Product_Name': product_name,
    'Discounted_prices': disc_prices,
    'Store_name': store_name,
    'Store_link': store_link,
    'Shipping_prices': shipping_prices,
    'Item_sold': item_sold,
    'Discount_percent': discount_percent,
    'Amount_saved': amount_saved,
    'actual_price': actual_prices
}

df = pd.DataFrame(data)
df.to_csv('ali_express_laptops.csv')

#### Data Cleaning

In [123]:
# Imporing dataset from saved csv-file
laptops = pd.read_csv('ali_express_laptops.csv', index_col=0)

laptops.head()

,Product_Name,Discounted_prices,Store_name,Store_link,Shipping_prices,Item_sold,Discount_percent,Amount_saved,actual_price
0,High-end Lenovo Laptop ThinkPad X1 Carbon 2022...,79150.50,HUWEI Factory Store,//www.aliexpress.com/store/836146,Free shipping,7,0.0,Free shipping,79150.50
1,High-end Lenovo Laptop ThinkPad P15v Gen 3 202...,60154.38,HUWEI Factory Store,//www.aliexpress.com/store/836146,Free shipping,6,0.0,Free shipping,60154.38
2,2022 Professional Lenovo Gaming Laptop Legion ...,60154.38,HUWEI Factory Store,//www.aliexpress.com/store/836146,Free shipping,5,0.0,Free shipping,60154.38
3,New A2681 Laptop LCD Screen Display Assembly F...,2229.42,MAC LCD Store,//www.aliexpress.com/store/1103570212,+shipping: 204.87,98,9.0,220.49,2449.91
4,2024 Ultrabook Laptop 15.6 inch AMD R7 4700U/R...,5509.82,YSJMNPC Official Store,//www.aliexpress.com/store/1100370457,Free shipping,25,44.0,4329.22,9839.04


In [124]:
laptops.shape

(600, 9)

#### Cleaning Data 

In [125]:
laptops.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 600 entries, 0 to 599
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Product_Name       600 non-null    object 
 1   Discounted_prices  600 non-null    float64
 2   Store_name         600 non-null    object 
 3   Store_link         600 non-null    object 
 4   Shipping_prices    461 non-null    object 
 5   Item_sold          543 non-null    object 
 6   Discount_percent   600 non-null    float64
 7   Amount_saved       549 non-null    object 
 8   actual_price       600 non-null    float64
dtypes: float64(3), object(6)
memory usage: 46.9+ KB


In [128]:
# Remove the double-slashes preceding the url
laptops['Store_link'] = laptops['Store_link'].str.replace('//', '')

# Convert the column to string first, then perform the replacements
laptops['Shipping_prices'] = laptops['Shipping_prices'].astype(str).str.replace('Free shipping', '0.00', regex=False).str.replace('\+shipping:', '', regex=True)

# Removing + from item sold figures
laptops['Item_sold'] = laptops['Item_sold'].astype(str).replace('+', '')

# Filling NaN = 0 for items sold
laptops['Item_sold'] = laptops['Item_sold'].fillna(0)

laptops['Amount_saved'] = laptops['Amount_saved'].astype(str).str.replace('Free shipping', '', regex=False).str.replace('\+shipping:', '', regex=True)

# Convert shipping price, items sold and amount saved to integer and float
laptops['Item_sold'] = pd.to_numeric(laptops['Item_sold'], errors='coerce').fillna(0).astype(int)

to_int = ['Shipping_prices','Amount_saved']
for i in to_int:
    laptops[i] = pd.to_numeric(laptops[i], errors='coerce').fillna(0).astype(float)


In [131]:
laptops.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 600 entries, 0 to 599
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Product_Name       600 non-null    object 
 1   Discounted_prices  600 non-null    float64
 2   Store_name         600 non-null    object 
 3   Store_link         600 non-null    object 
 4   Shipping_prices    600 non-null    float64
 5   Item_sold          600 non-null    int64  
 6   Discount_percent   600 non-null    float64
 7   Amount_saved       600 non-null    float64
 8   actual_price       600 non-null    float64
dtypes: float64(5), int64(1), object(3)
memory usage: 46.9+ KB


In [132]:

laptops.head()

,Product_Name,Discounted_prices,Store_name,Store_link,Shipping_prices,Item_sold,Discount_percent,Amount_saved,actual_price
0,High-end Lenovo Laptop ThinkPad X1 Carbon 2022...,79150.50,HUWEI Factory Store,www.aliexpress.com/store/836146,0.00,7,0.0,0.00,79150.50
1,High-end Lenovo Laptop ThinkPad P15v Gen 3 202...,60154.38,HUWEI Factory Store,www.aliexpress.com/store/836146,0.00,6,0.0,0.00,60154.38
2,2022 Professional Lenovo Gaming Laptop Legion ...,60154.38,HUWEI Factory Store,www.aliexpress.com/store/836146,0.00,5,0.0,0.00,60154.38
3,New A2681 Laptop LCD Screen Display Assembly F...,2229.42,MAC LCD Store,www.aliexpress.com/store/1103570212,204.87,98,9.0,220.49,2449.91
4,2024 Ultrabook Laptop 15.6 inch AMD R7 4700U/R...,5509.82,YSJMNPC Official Store,www.aliexpress.com/store/1100370457,0.00,25,44.0,4329.22,9839.04


In [145]:
# Product Table
product = laptops[['Product_Name', 'actual_price', 'Discount_percent']].copy().drop_duplicates().reset_index(drop=True)
product.index.name = 'product_id'
product = product.reset_index()
product.columns = ['product_id', 'product_name', 'actual_price', 'percent_discount']
product.head()

,product_id,product_name,actual_price,percent_discount
0,0,High-end Lenovo Laptop ThinkPad X1 Carbon 2022...,79150.50,0.0
1,1,High-end Lenovo Laptop ThinkPad P15v Gen 3 202...,60154.38,0.0
2,2,2022 Professional Lenovo Gaming Laptop Legion ...,60154.38,0.0
3,3,New A2681 Laptop LCD Screen Display Assembly F...,2449.91,9.0
4,4,2024 Ultrabook Laptop 15.6 inch AMD R7 4700U/R...,9839.04,44.0


In [144]:
# Store Table
store = laptops[['Store_name', 'Store_link']].copy().drop_duplicates().reset_index(drop=True)
store.index.name = 'store_id'
store = store.reset_index()
store.columns = ['store_id', 'store_name', 'store_link']
store.head()

,store_id,store_name,store_link
0,0,HUWEI Factory Store,www.aliexpress.com/store/836146
1,1,MAC LCD Store,www.aliexpress.com/store/1103570212
2,2,YSJMNPC Official Store,www.aliexpress.com/store/1100370457
3,3,NWNLAP Store,www.aliexpress.com/store/1518128
4,4,Shop1103665052 Store,www.aliexpress.com/store/1103665052


In [147]:
# Sales Table
sales = laptops[['Item_sold', 'Discounted_prices', 'Amount_saved', 'Shipping_prices']].copy().drop_duplicates().reset_index(drop=True)
sales.index.name = 'sales_id'
sales = sales.reset_index()
sales.columns = ['sales_id', 'quantity_sold', 'discounted_price', 'amount_saved', 'shipping_fee']
sales.head()

,sales_id,quantity_sold,discounted_price,amount_saved,shipping_fee
0,0,7,79150.50,0.00,0.00
1,1,6,60154.38,0.00,0.00
2,2,5,60154.38,0.00,0.00
3,3,98,2229.42,220.49,204.87
4,4,25,5509.82,4329.22,0.00


#### Saving Tables

In [173]:
product.to_csv('product.csv', index=False)
store.to_csv('store.csv', index=False)
sales.to_csv('sales.csv', index=False)

#### Creating PostgresSQL Database

In [177]:
# Create database connection
def get_db_connection():
    connection = psycopg2.connect(
        host = 'localhost',
        database = 'laptops_db',
        user = 'postgres',
        password = 'admin'
    )
    return connection

def create_tables():
    conn = get_db_connection()
    cursor = conn.cursor()

    create_table_query = ''' 
            CREATE SCHEMA IF NOT EXISTS laptopsSCHEMA;
                DROP TABLE IF EXISTS laptopsSCHEMA.sales CASCADE;
                DROP TABLE IF EXISTS laptopsSCHEMA.stores CASCADE;
                DROP TABLE IF EXISTS laptopsSCHEMA.product CASCADE;
                
                

            CREATE TABLE IF NOT EXISTS laptopsSCHEMA.product(
                product_id SERIAL PRIMARY KEY NOT NULL,
                product_name TEXT NOT NULL, 
                actual_price FLOAT NOT NULL, 
                percent_discount FLOAT
            );

            CREATE TABLE IF NOT EXISTS laptopsSCHEMA.stores(
                store_id SERIAL PRIMARY KEY NOT NULL,
                store_name VARCHAR(500) NOT NULL,
                store_link VARCHAR(500) UNIQUE NOT NULL
            );

            CREATE TABLE IF NOT EXISTS laptopsSCHEMA.sales(
                sales_id SERIAL PRIMARY KEY NOT NULL,
                product_id INTEGER,
                store_id INTEGER,
                quantity_sold INTEGER NOT NULL,
                discounted_price FLOAT,
                amount_saved FLOAT,
                shipping_fee FLOAT,
                FOREIGN KEY (product_id) REFERENCES laptopsSCHEMA.product(product_id),
                FOREIGN KEY (store_id) REFERENCES laptopsSCHEMA.stores(store_id)
            );
'''
    cursor.execute(create_table_query)
    conn.commit()
    cursor.close()
    conn.close()

create_tables()
